## Interval Analysis

In [1]:
# !pip install tensorboardX
# !pip install bound-propagation


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()


Sequential(
  (0): Normalize()
  (1): Net(
    (fc1): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [2]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [3]:
train_model(model, 15)
test_model(model)

Epoch 1/15, Loss: 2.012
Epoch 2/15, Loss: 1.729
Epoch 3/15, Loss: 1.627
Epoch 4/15, Loss: 1.596
Epoch 5/15, Loss: 1.582
Epoch 6/15, Loss: 1.572
Epoch 7/15, Loss: 1.566
Epoch 8/15, Loss: 1.561
Epoch 9/15, Loss: 1.557
Epoch 10/15, Loss: 1.553
Epoch 11/15, Loss: 1.550
Epoch 12/15, Loss: 1.547
Epoch 13/15, Loss: 1.545
Epoch 14/15, Loss: 1.542
Epoch 15/15, Loss: 1.540
Accuracy on images: 93.43


### Write the interval analysis for the simple model

In [13]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation

from bound_propagation import BoundModelFactory, BoundModule, IntervalBounds, HyperRectangle
from bound_propagation.sequential import BoundSequential

class BoundNet(BoundModule):
    def __init__(self, model, factory, **kwargs):
        super().__init__(model, factory, **kwargs)
        self.fc1 = factory.build(model.fc1)
        self.relu = factory.build(nn.ReLU())
        self.fc2 = factory.build(model.fc2)
        self.softmax = factory.build(nn.Softmax(dim=-1))

    def propagate_size(self, in_size):
        # We have to flatten the input here to match the first linear layer
        size = (in_size[0], 784)
        size = self.fc1.propagate_size(size)
        size = self.relu.propagate_size(size)
        size = self.fc2.propagate_size(size)
        size = self.softmax.propagate_size(size)
        return size

    def ibp_forward(self, bounds, save_relaxation=False, save_input_bounds=False):
        # Again have to flatten the bounds to match the first linear layer
        lower = bounds.lower.view(bounds.lower.size(0), -1)
        upper = bounds.upper.view(bounds.upper.size(0), -1)
        bounds = IntervalBounds(bounds.region, lower, upper)

        bounds = self.fc1.ibp_forward(bounds, save_relaxation, save_input_bounds)
        bounds = self.relu.ibp_forward(bounds, save_relaxation, save_input_bounds)
        bounds = self.fc2.ibp_forward(bounds, save_relaxation, save_input_bounds)
        bounds = self.softmax.ibp_forward(bounds, save_relaxation, save_input_bounds)
        return bounds

    @property
    def need_relaxation(self):
        # Not needed for ibp
        raise NotImplementedError()

    def clear_relaxation(self):
        # Not needed for ibp
        raise NotImplementedError()

    def backward_relaxation(self, region):
        # Not needed for ibp
        raise NotImplementedError()

    def crown_backward(self, linear_bounds, optimize):
        # Not needed for ibp
        raise NotImplementedError()


class BoundNormalize(BoundModule):
    def __init__(self, model, factory, **kwargs):
        super().__init__(model, factory, **kwargs)
        self.mean = torch.tensor([0.1307], device=device)
        self.std = torch.tensor([0.3081], device=device)

    def propagate_size(self, in_size):
        return in_size

    def ibp_forward(self, bounds, save_relaxation=False, save_input_bounds=False):
        lower = (bounds.lower - self.mean) / self.std
        upper = (bounds.upper - self.mean) / self.std
        return IntervalBounds(bounds.region, lower, upper)

    @property
    def need_relaxation(self):
        # Not needed for ibp
        return False

    def clear_relaxation(self):
        # Not needed for ibp
        raise NotImplementedError()

    def backward_relaxation(self, region):
        # Not needed for ibp
        raise NotImplementedError()

    def crown_backward(self, linear_bounds, optimize):
        # Not needed for ibp
        raise NotImplementedError()


class BoundSoftmax(BoundModule):
    def __init__(self, model, factory, **kwargs):
        super().__init__(model, factory, **kwargs)

    def propagate_size(self, in_size):
        return in_size

    def ibp_forward(self, bounds, save_relaxation=False, save_input_bounds=False):
        lower = F.softmax(bounds.lower, dim=-1)
        upper = F.softmax(bounds.upper, dim=-1)
        return IntervalBounds(bounds.region, lower, upper)

    @property
    def need_relaxation(self):
        # Not needed for ibp
        return False

    def clear_relaxation(self):
        # Not needed for ibp
        raise NotImplementedError()

    def backward_relaxation(self, region):
        # Not needed for ibp
        raise NotImplementedError()

    def crown_backward(self, linear_bounds, optimize):
        # Not needed for ibp
        raise NotImplementedError()



In [14]:
# Register all custom bound modules
factory = BoundModelFactory()
factory.register(nn.Sequential, BoundSequential)
factory.register(Normalize, BoundNormalize)
factory.register(Net, BoundNet)
factory.register(nn.Softmax, BoundSoftmax)
factory_model = factory.build(model)

In [15]:
# Perfom ibp analysis
# Loop over different epsilon values
for i in range(1, 11):
    eps = i * 0.01
    verified_count = 0
    total_count = 0

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get input and output bounds with current epsilon
        input_bounds = HyperRectangle.from_eps(images, eps)
        output_bounds = factory_model.ibp(input_bounds)

        # Loop through each sample in the batch to verify robustness
        for i, true_class in enumerate(labels):
            lower = output_bounds.lower[i]
            upper = output_bounds.upper[i]

            # Check if lower bound of true class is greater than upper bounds of all other classes
            other_start = upper[:true_class]
            other_end = upper[true_class+1:]
            max_other = torch.max(torch.cat((other_start, other_end)))
            if lower[true_class] > max_other:
                verified_count += 1
            total_count += 1

    verified_accuracy = 100 * verified_count / total_count
    print(f"Verified accuracy with eps={eps:.2f}: {verified_accuracy:.2f}%")


Verified accuracy with eps=0.01: 93.36%
Verified accuracy with eps=0.02: 92.96%
Verified accuracy with eps=0.03: 92.05%
Verified accuracy with eps=0.04: 90.03%
Verified accuracy with eps=0.05: 86.32%
Verified accuracy with eps=0.06: 80.63%
Verified accuracy with eps=0.07: 74.21%
Verified accuracy with eps=0.08: 67.53%
Verified accuracy with eps=0.09: 61.43%
Verified accuracy with eps=0.10: 55.92%
